In [655]:
class Item:
    def __init__(self, item: str, utility: int):
        self.item = item
        self.utility = utility

    def __repr__(self):
        # return f"({self.item},{self.utility})"
        return f"{self.item}"
    
    
    # Định nghĩa phương thức so sánh __eq__
    def __eq__(self, other):
        if isinstance(other, Item):
            return self.item == other.item  # So sánh dựa trên giá trị 'item'
        return False

    # Định nghĩa phương thức băm __hash__ để có thể sử dụng trong set hoặc dict
    def __hash__(self):
        return hash(self.item)  # Băm dựa trên giá trị 'item'

In [656]:
class Transaction:
    def __init__(self, id: int, items_quantities: dict):
        if any(q <= 0 for q in items_quantities.values()):
            raise ValueError(f"Quantities in trans{id} must be positive integers.")
        self.id = id
        self.items_quantities = items_quantities

    def __repr__(self):
        return (f"(tid = {self.id}, frequencies = {self.items_quantities})")

In [657]:
a = Item('A', 1)
b = Item('B', 2)
c = Item('C', 5)
d = Item('D', 2)
e = Item('E', 3)


items = [a, b, c, d, e]

trans1 = Transaction(1, dict(zip([a, b, c, d, e], [5, 3, 2, 4, 3])))
trans2 = Transaction(2, dict(zip([a, b, c, d], [6, 4, 3, 5])))
trans3 = Transaction(3, dict(zip([a, d, e], [4, 2, 1])))
trans4 = Transaction(4, dict(zip([a, e], [7, 6])))
trans5 = Transaction(5, dict(zip([a, b, d, e], [5, 4, 4, 3])))
trans6 = Transaction(6, dict(zip([b,e], [8, 7])))


database = [trans1, trans2, trans3, trans4, trans5, trans6]

database

[(tid = 1, frequencies = {A: 5, B: 3, C: 2, D: 4, E: 3}),
 (tid = 2, frequencies = {A: 6, B: 4, C: 3, D: 5}),
 (tid = 3, frequencies = {A: 4, D: 2, E: 1}),
 (tid = 4, frequencies = {A: 7, E: 6}),
 (tid = 5, frequencies = {A: 5, B: 4, D: 4, E: 3}),
 (tid = 6, frequencies = {B: 8, E: 7})]

In [658]:
trans1.items_quantities.keys()
trans1.items_quantities.values()
print(trans6.items_quantities.get(d))

None


In [659]:
def calculate_positive_utility_of_item_set_in_trans(items: set[Item], trans: Transaction):
    pu = 0
    for item in items:
        utility = item.utility 
        if(utility > 0):
            quantity = trans.items_quantities.get(item);
            if(quantity != None):
                pu += item.utility * quantity
    return pu;

In [660]:
def calculate_positive_utility_of_item_set_in_database(items: set[Item], database: list[Transaction]):
    pu = 0
    for trans in database:
        if items.issubset(trans.items_quantities.keys()):
            pu += calculate_positive_utility_of_item_set_in_trans(items, trans)
    return pu

In [661]:
def calculate_negative_utility_of_item_set_in_trans(items: set[Item], trans: Transaction):
    nu = 0
    for item in items:
        utility = item.utility 
        if(utility < 0):
            quantity = trans.items_quantities.get(item);
            if(quantity != None):
                nu += item.utility * quantity
    return nu;

In [662]:
def calculate_negative_utility_of_item_set_in_database(items: set[Item], database: list[Transaction]):
    nu = 0
    for trans in database:
        if items.issubset(trans.items_quantities.keys()):
            nu += calculate_negative_utility_of_item_set_in_trans(items, trans)
    return nu

???? negative utility of transaction

In [663]:
def calculate_negative_utility_of_transaction(trans: Transaction):
    nu = 0;
    items : set[Item] = trans.items_quantities.keys()
    for item in items:
        if(item.utility < 0):
            nu += trans.items_quantities.get(item) * item.utility
    return nu;

???? positive utility of transaction

In [664]:
def calculate_positive_utility_of_transaction(trans: Transaction):
    pu = 0;
    items : set[Item] = trans.items_quantities.keys()
    for item in items:
        if(item.utility > 0):
            pu += trans.items_quantities.get(item) * item.utility
    return pu;

In [665]:
def calculate_positive_utility_of_database(database: list[Transaction]):
    pu = 0;
    for trans in database:
        pu += calculate_positive_utility_of_transaction(trans)
    return pu

In [666]:
def calculate_negative_utility_of_database(database: list[Transaction]):
    pu = 0;
    for trans in database:
        pu += calculate_negative_utility_of_transaction(trans)
    return pu

In [673]:
calculate_positive_utility_of_database(database)

0

In [667]:
def calculate_twu(item):
    return;

In [668]:
def calculate_max_per(item):
    return;

In [669]:
def calculate_avg_per(item):
    return;

In [670]:
def preparation_procedure(database, minUtility, minPer, maxPer, minAvg, maxAvg):
    return;

In [671]:
list1 = [a, b, c]
list2 = [a, b, c, d]
